In [1]:
import os
import argparse

from keras.datasets import mnist
from keras.optimizers import RMSprop

import sys
sys.path.append('/home/hm0/Research/molecules/molecules_git/build/lib')

from molecules.ml.unsupervised import EncoderConvolution2D
from molecules.ml.unsupervised import DecoderConvolution2D
from molecules.ml.unsupervised import VAE

Using TensorFlow backend.


In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
img_rows = img_cols = 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [5]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

encoder = EncoderConvolution2D(input_shape=input_shape)

encoder._get_final_conv_params()
num_conv_params = encoder.total_conv_params
encode_conv_shape = encoder.final_conv_shape

decoder = DecoderConvolution2D(output_shape=input_shape,
                               enc_conv_params=num_conv_params,
                               enc_conv_shape=encode_conv_shape)

cvae = VAE(input_shape=input_shape,
           latent_dim=3,
           encoder=encoder,
           decoder=decoder,
           optimizer=optimizer)


In [9]:
cvae.summary()

Convolutional Variational Autoencoder

Convolutional Encoder:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense

In [10]:
cvae.train(x_train, validation_data=x_test, batch_size=128, epochs=10)


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 22s 371us/step - loss: 171.8421 - val_loss: 150.5499
Epoch 2/10
60000/60000 [==============================] - 19s 314us/step - loss: 140.5181 - val_loss: 138.2347
Epoch 3/10
60000/60000 [==============================] - 20s 338us/step - loss: 134.3325 - val_loss: 135.1584
Epoch 4/10
60000/60000 [==============================] - 20s 338us/step - loss: 131.0672 - val_loss: 132.4982
Epoch 5/10
60000/60000 [==============================] - 19s 323us/step - loss: 128.7918 - val_loss: 130.3798
Epoch 6/10
60000/60000 [==============================] - 20s 338us/step - loss: 127.1508 - val_loss: 129.8448
Epoch 7/10
60000/60000 [==============================] - 20s 337us/step - loss: 125.7762 - val_loss: 130.6703
Epoch 8/10
60000/60000 [==============================] - 19s 323us/step - loss: 124.7479 - val_loss: 127.6623
Epoch 9/10
60000/60000 [==============================] - 20s 

In [ ]:
weight_path = os.path.join(args.weight_path, 'cvae_mnist.h5')
cvae.save_weights(weight_path)